<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_15/S15_pl_lightning_En_Fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


Install packages

In [1]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull
#!cd ../
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git -C dl_hub pull
!git pull
#!cd ../

!pip install --quiet "torchinfo" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts" "torchtext" "datasets" "tokenizers"
!pip install --quiet "prettytable"
!pip install --quiet "torchsummary"
!pip install --quiet "torch_lr_finder"
!pip install --quiet "grad-cam"
!pip install --quiet "gradio"

Cloning into 'ERA_V1'...
remote: Enumerating objects: 1412, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 1412 (delta 244), reused 426 (delta 152), pack-reused 821
Receiving objects: 100% (1412/1412), 241.99 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (689/689), done.
Already up to date.
Cloning into 'dl_hub'...
remote: Enumerating objects: 566, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 566 (delta 211), reused 283 (delta 183), pack-reused 248
Receiving objects: 100% (566/566), 144.47 KiB | 8.50 MiB/s, done.
Resolving deltas: 100% (355/355), done.
Already up to date.
fatal: not a git repository (or any of the parent directories): .git
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import sys
sys.path.append("ERA_V1/session_15")
sys.path.append("dl_hub")

In [ ]:
!git -C dl_hub pull


Already up to date.


In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
from dl_hub.Transformer.config import get_config
from dl_hub.Transformer.PL_data_module import OpusDataModule
from dl_hub.Transformer.PL_model import LitTransformer
from dl_hub.Transformer.PL_main import train_transformer



In [ ]:
#from config import get_config
#from PL_data_module import OpusDataModule
#from PL_model import LitTransformer
#from PL_main import train_transformer


**Finetune config params**

In [ ]:
cfg = get_config()
cfg["parameter_sharing"] = True
cfg["d_ff"] = 1024
cfg["batch_size"] = 48
cfg["d_model"] = 512
cfg["preload"] = None
cfg["num_epochs"] = 40
cfg["ds_mode"] = 'Online'
cfg["lang_tgt"] = 'fr'
cfg["save_ds_to_disk"] = False
cfg["experiment_name"] = f"tmodel_dynamic_pad_150_tokens_bs_{cfg['batch_size']}_amp_ps_dff_{cfg['d_ff']}"



**Mixed precision training**

In [ ]:
torch.cuda.amp.autocast(enabled=True)

**Define data module**

In [ ]:
opus_dm = OpusDataModule(cfg)
opus_dm.prepare_data()
opus_dm.setup()
train_data_loader = opus_dm.train_dataloader()
#pp = next(iter(train_data_loader))

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

**Define PL Model**

In [ ]:
tokenizer_src = opus_dm.tokenizer_src
tokenizer_tgt = opus_dm.tokenizer_tgt
loss_criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
transformer_model = LitTransformer(cfg, loss_criterion, tokenizer_src, tokenizer_tgt, num_validation_examples=2, epochs = cfg['num_epochs'])
optimizer = torch.optim.Adam(transformer_model.model.parameters(), lr=cfg['lr'], eps=1e-9)
transformer_model.set_optimizer(optimizer)

scheduler = OneCycleLR(
    optimizer,
    max_lr=1E-3,
    steps_per_epoch=len(train_data_loader),
    epochs=cfg['num_epochs'],
    pct_start=5/cfg['num_epochs'],
    div_factor=10,
    three_phase=True,
    final_div_factor=10,
    anneal_strategy='linear'
)
transformer_model.set_scheduler_dict(scheduler, freq = 'step')

Total Parameters: 61847890


In [ ]:
train_transformer(transformer_model, opus_dm, cfg, epochs=cfg['num_epochs'])

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: runs/tmodel_dynamic_pad_150_tokens_bs_48_amp_ps_dff_1024
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | loss_criterion | CrossEntropyLoss | 0     
1 | model          | Transformer      | 61.8 M
----------------------------------------------------
61.8 M    Trainable params
0         Non-trainable params
61.8 M    Total params
123.696   Total estimated model params size (MB)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: He was in the same lounge I had to cross in order to escape.
    TARGET: Il était dans ce salon que je devais traverser pour fuir.
 PREDICTED: Il était à la fois que je me à la maison .
*****************************************

*****************************************
    SOURCE: He was no longer my equal, he was the Man of the Waters, the Spirit of the Seas.
    TARGET: Ce n'était plus mon semblable, c'était l'homme des eaux, le génie des mers.
 PREDICTED: Il n ’ était plus plus que je l ’ de la rue de la rue de la maison , le .
*****************************************



/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:61: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:61: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:61: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(


...

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: All the valley at my right hand was full of pasture- fields, and cornfields, and wood; and a glittering stream ran zig-zag through the varied shades of green, the mellowing grain, the sombre woodland, the clear and sunny lea.
    TARGET: À ma droite, la vallée était remplie de pâturages, de bois et de champs de grains; un ruisseau tortueux coulait au milieu du feuillage aux teintes variées, des champs mûrs, de sombres forêts et des prairies éclairées par le soleil.
 PREDICTED: Toute la vallée à ma droite était à plein de bateaux à bateaux , et de très en bois . Un ruisseau coulait à travers les ombres brusques ombres du vert genre vert et le temps triste , le léger et .
*****************************************

*****************************************
    SOURCE: Two old friends mustn't fall out over such a--well, there, I won't say it.
    TARGET: Deux vieux amis qui se prennent corps à corps pour une... Non, je ne le dirai pas.


`Trainer.fit` stopped: `max_epochs=40` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

*****************************************
    SOURCE: Her pretty Djali followed her.
    TARGET: Sa jolie Djali la suivait.
 PREDICTED: Sa jolie Djali la suivait .
*****************************************

*****************************************
    SOURCE: This unsettled demeanour alarmed him, for he guessed, in part, what her struggles and troubles must be like.
    TARGET: L'indécision où il la voyait l'effrayait, car il devinait en partie ses luttes et ses troubles.
 PREDICTED: Cet entêtement l ' effraya , car il devina les jours , quelles que ses chagrins pouvaient être comme chacun .
*****************************************



**Load trained model**

In [ ]:
model_path = "/content/gdrive/MyDrive/trained_models/Transformer/transformer_model_param_sharing_1024_dff.ckpt"
trained_transformer_model = LitTransformer.load_from_checkpoint(model_path, map_location=torch.device('cuda'), config=cfg, loss_criterion=loss_criterion, tokenizer_src = tokenizer_src, tokenizer_tgt = tokenizer_tgt )

Total Parameters: 61847890


**Run Inference on sentences**

In [ ]:
def encode_input_sentence(input, tokenizer_src, max_seq_len) -> torch.tensor:
    encoder_input_tokens = tokenizer_src.encode(input).ids
    sos_token = torch.tensor([tokenizer_src.token_to_id("[SOS]")], dtype=torch.int64)
    eos_token = torch.tensor([tokenizer_src.token_to_id("[EOS]")], dtype=torch.int64)
    pad_token = torch.tensor([tokenizer_src.token_to_id("[PAD]")], dtype=torch.int64)
    # Add sos, eos and padding to each sentence
    enc_num_padding_tokens = max_seq_len - len(encoder_input_tokens) - 2 # we will add <s> and </s>

    encoder_input = torch.cat(
        [
            sos_token,
            torch.tensor(encoder_input_tokens, dtype=torch.int64),
            eos_token,
            torch.tensor([pad_token]*enc_num_padding_tokens, dtype=torch.int64)
        ],
        dim = 0)
    encoder_mask = (encoder_input != pad_token).unsqueeze(0).unsqueeze(0).int()
    return encoder_input, encoder_mask

def translate_en_to_fr(input, opus_dm, cfg):
    encoder_input, encoder_mask = encode_input_sentence(input, opus_dm.tokenizer_src, cfg['seq_len'])
    output = trained_transformer_model.greedy_decode(encoder_input, encoder_mask)
    model_out_text = opus_dm.tokenizer_tgt.decode(output.detach().cpu().numpy())
    return model_out_text



In [ ]:
input = "The firm resolution to commit another murder somewhat calmed them."
output = translate_en_to_fr(input, opus_dm, cfg)

print(output)

La résolution bien arrêtée d ' un assassinat les calma un peu .


In [8]:
#pip install -q -U ngrok
!kill 2330
%load_ext tensorboard
%tensorboard --logdir='ERA_V1/session_15/runs'

/bin/bash: line 1: kill: (2330) - No such process
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3905), started 0:02:33 ago. (Use '!kill 3905' to kill it.)

<IPython.core.display.Javascript object>